In [59]:
import pandas as pd
from math import sqrt
import random

In [60]:
movies_df = pd.read_csv('movies.csv')
movies_df['title'] = movies_df['title'].str.replace(r' \(\d{4}\)', '')
ratings_df = pd.read_csv('ratings.csv')
print(movies_df.info())
movies_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
None


C:\Users\bryan\AppData\Local\Temp\ipykernel_7272\1786508211.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df['title'].str.replace(r' \(\d{4}\)', '')


,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy


In [61]:

userInput = [{'title':'Breakfast Club, The', 'rating':random.randint(1, 5)},
             {'title':'Toy Story', 'rating':random.randint(1, 5)},
             {'title':'Jumanji', 'rating':random.randint(1, 5)},
             {'title':'Pulp Fiction', 'rating':random.randint(1, 5)},
             {'title':'Akira', 'rating':random.randint(1, 5)}]
inputMovies = pd.DataFrame(userInput)
print(inputMovies)

                 title  rating
0  Breakfast Club, The       3
1            Toy Story       1
2              Jumanji       4
3         Pulp Fiction       3
4                Akira       5


In [62]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies[['movieId','title','rating']]
print(inputMovies)

   movieId                title  rating
0        1            Toy Story       1
1        2              Jumanji       4
2      296         Pulp Fiction       3
3     1274                Akira       5
4     1968  Breakfast Club, The       3


In [63]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
print(userSubset.groupby('movieId').count())

         userId  rating  timestamp
movieId                           
1           215     215        215
2           110     110        110
296         307     307        307
1274         39      39         39
1968        113     113        113


In [64]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

def take_5_elem(x):
    # print (len(x[1]))
    return len(x[1])
    

#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup, key=take_5_elem, reverse=True)

userSubsetGroup = userSubsetGroup[0:100]
print(userSubsetGroup[0:5])

[(91,        userId  movieId  rating   timestamp
14121      91        1     4.0  1112713037
14122      91        2     3.0  1112713392
14173      91      296     4.5  1112711264
14316      91     1274     5.0  1112713057
14383      91     1968     3.0  1112713409), (177,        userId  movieId  rating   timestamp
24900     177        1     5.0  1435533535
24901     177        2     3.5  1435534109
24930     177      296     5.0  1435530409
25069     177     1274     2.0  1435535036
25129     177     1968     3.5  1435534080), (219,        userId  movieId  rating   timestamp
31524     219        1     3.5  1194681084
31525     219        2     2.5  1194740185
31554     219      296     4.0  1198522553
31628     219     1274     2.5  1194686351
31680     219     1968     3.0  1194931899), (274,        userId  movieId  rating   timestamp
39229     274        1     4.0  1171410158
39230     274        2     3.5  1171934785
39288     274      296     5.0  1171493995
39448     274     1274  

C:\Users\bryan\AppData\Local\Temp\ipykernel_7272\2911040200.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup = sorted(userSubsetGroup, key=take_5_elem, reverse=True)


In [65]:
pearsonCorrelationDict = {}

for name, group in userSubsetGroup:

    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')

    nRatings = len(group)

    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]

    tempRatingList = temp_df['rating'].tolist()
   
    tempGroupList = group['rating'].tolist()
   
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
print(pearsonDF.head())

   similarityIndex  userId
0         0.207289      91
1        -0.846114     177
2        -0.672214     219
3        -0.184637     274
4         0.164887     298


In [66]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
print(topUsers.head())

    similarityIndex  userId
43         1.000000     132
90         1.000000     562
65         0.944911     323
95         0.944911     604
74         0.866025     391


In [67]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
print(topUsersRating.head(100))

    similarityIndex  userId  movieId  rating   timestamp
0               1.0     132        1     2.0  1157921785
1               1.0     132       17     3.0  1157922698
2               1.0     132       29     2.0  1157924165
3               1.0     132       32     3.0  1329983726
4               1.0     132       34     1.5  1157921395
..              ...     ...      ...     ...         ...
95              1.0     132     1258     4.0  1157919831
96              1.0     132     1259     3.0  1157922770
97              1.0     132     1270     3.5  1157921202
98              1.0     132     1271     3.5  1157923484
99              1.0     132     1296     2.5  1157978896

[100 rows x 5 columns]


In [68]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
print(topUsersRating.head())

   similarityIndex  userId  movieId  rating   timestamp  weightedRating
0              1.0     132        1     2.0  1157921785             2.0
1              1.0     132       17     3.0  1157922698             3.0
2              1.0     132       29     2.0  1157924165             2.0
3              1.0     132       32     3.0  1329983726             3.0
4              1.0     132       34     1.5  1157921395             1.5


In [69]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
print(tempTopUsersRating.head())

         sum_similarityIndex  sum_weightedRating
movieId                                         
1                  17.385975           55.229491
2                   8.839400           33.085589
3                   2.455434            7.314017
4                   0.542857            0.814286
5                   3.835931           12.952072


In [70]:
recommendation_df = pd.DataFrame()

recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
print(recommendation_df.head(10))

         weighted average recommendation score  movieId
movieId                                                
1                                     3.176669        1
2                                     3.742968        2
3                                     2.978707        3
4                                     1.500000        4
5                                     3.376514        5
6                                     3.995283        6
7                                     2.890128        7
9                                     1.500000        9
10                                    3.478992       10
11                                    3.209273       11


In [71]:
recommendation_df = recommendation_df[recommendation_df['weighted average recommendation score'] == 5]
print(recommendation_df)

         weighted average recommendation score  movieId
movieId                                                
85                                         5.0       85
187                                        5.0      187
456                                        5.0      456
633                                        5.0      633
945                                        5.0      945
...                                        ...      ...
162414                                     5.0   162414
167064                                     5.0   167064
179135                                     5.0   179135
184245                                     5.0   184245
187717                                     5.0   187717

[82 rows x 2 columns]


In [72]:
recommended_movie=movies_df.loc[movies_df['movieId'].isin(recommendation_df['movieId'])]

recommended_movie=recommended_movie.loc[~recommended_movie.movieId.isin(userSubset['movieId'])]

print(recommended_movie)

      movieId                      title                  genres
76         85         Angels and Insects           Drama|Romance
158       187                 Party Girl                  Comedy
397       456                      Fresh    Crime|Drama|Thriller
536       633            Denise Calls Up                  Comedy
725       945                    Top Hat  Comedy|Musical|Romance
...       ...                        ...                     ...
9367   162414                  Moonlight                   Drama
9443   167064        I Am Not Your Negro             Documentary
9634   179135             Blue Planet II             Documentary
9688   184245           De platte jungle             Documentary
9711   187717  Won't You Be My Neighbor?             Documentary

[82 rows x 3 columns]
